In [ ]:
import keras, numpy as np, matplotlib as mpl, matplotlib.pyplot as plt, tqdm
import os.path
import time
%matplotlib inline

In [ ]:
import nnUtils

In [ ]:
import sklearn, sklearn.model_selection

In [ ]:
datadir = os.path.join(os.path.expanduser('~'), 'data2', 'behavioralCloning')

In [ ]:
# data = np.load(os.path.join(datadir, 'multiData-mouseForwardFuller.npz'))
# X = data['X']
# Y = data['Y'][:, 0].reshape(-1, 1)

In [ ]:
!ls ~/data2/behavioralCloning/*.zip

In [ ]:
import loadData, models

from importlib import reload
reload(loadData);
reload(models);

In [ ]:
dataGenerator = loadData.DeemphasizedZeroDataGenerator(
    [
        os.path.join(loadData.HOME, 'data2', 'behavioralCloning', p)
        for p in (
#             'data_provided.zip',
            'mouseForward.zip',
            'mouseReverse.zip',
            'longCarefulForward.zip',
            'longCarefulReverse.zip',
#             'dirtSidesForward.zip',
#             'dirtSidesReverse.zip',
#             'data_provided.zip',
        )
    ],
)

In [ ]:
NEPOCH = 6

In [ ]:
kernel_regularizer = keras.regularizers.l2(0.0001)
#bias_regularizer = keras.regularizers.l2(0.01)

modelname = ('deemZero-mouseLongCare-kl2%.1g-%depoch-fulldata' % (
    kernel_regularizer.l2, NEPOCH
)).replace('0.', 'p')

In [ ]:
%%time
Model = models.Nvidia

model = Model(
    1, dataGenerator.sampleImageShape,
    optimizer=keras.optimizers.Nadam(
        lr=0.0001, #beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)
    ),
    #kernel_regularizer=kernel_regularizer,
    #bias_regularizer=bias_regularizer,
)

In [ ]:
%%time
trainGen = dataGenerator.generate(epochSubsampling=1)
validGen = dataGenerator.generate(validation=True)
history = nnUtils.fitModelWithDataGenerator(model, dataGenerator, modelname, epochs=NEPOCH, max_queue_size=100)

In [ ]:
fpath = os.path.join(datadir, '%s.h5' % modelname)
print('Saving to', fpath)
model.save(fpath)

In [ ]:
!recent /home/tsbertalan/data2/behavioralCloning/ | head -n 4

In [ ]:
reload(loadData)

trainIndices, validationIndices = dataGenerator._indices
for ind, lab in zip(dataGenerator._indices, ('train', 'validation')):
    
    imgs = []
    responses = []
    for k in range(ind[0], ind[0]+1000):
        try:
            image, response = dataGenerator.getImageResponse(k)
            imgs.append(image.reshape((1, *image.shape)))
            responses.append(response)
        except KeyError:
            break
    X = np.vstack(imgs)
    Y = np.vstack(responses)

    pred = model.predict(X)
    pred.shape, Y.shape

    from collections import deque
    def runningMean(x, N):
        y = []
        hist = deque(maxlen=N)
        for a in x:
            hist.append(a)
            y.append(np.mean(hist))
        return np.array(y)
    
    ind = np.copy(ind)
    ind.sort()
    fig, ax = plt.subplots()
    if lab == 'train':
        start = 580
        end = start + 160
    else:
        start = 243
        end = start + 60
    samples = range(start, end)
    
    ax.plot(samples, Y[start:end], label=r'truth $\theta$', color='black')
    ax.plot(
        samples, 
        pred[start:end],
        label=r'predictions $\hat\theta$', color='magenta',
        #s=10,
    )
    filtersize = 6
    scale = 1
    smoothed = scale * runningMean(pred, filtersize)
    ax.plot(
        samples, smoothed[start:end], 
        label=r'$\hat\rho = %.1f \cdot box_{%d}(\hat\theta)$' % (scale, filtersize),
        #alpha=.5,
        color='green',
    )
    
    # Add some insets of the input.
    ninset =6
    w = h = 1./(ninset+2)
    for xloc, i in zip(
        np.linspace(w, 1-2*w, ninset),
        np.linspace(start, end-1, ninset).astype(int)
    ):
        l = xloc; b = .1100
        inset = fig.add_axes([l, b, w, h])
        loadData.showxy(
            X[i], Y[i], y2=pred[i], ax=inset, 
            l1kwargs=dict(color='white', linewidth=2), l2kwargs=dict(color='magenta', linewidth=2, alpha=.5)
        )
        inset.set_title(
            '$t=%d$' % samples[i-start]
#             + '\n' + 
#             r'$\theta=%.2g$' % (Y[i],)
#             + '\n' +
#             r'$\hat\theta=%.2g$' % (pred[i],),
            ,fontsize=10,
        )
    
    ax.set_xlabel('time [samples]')
    ax.set_ylabel('angle [radians]')
    ax.set_title(lab)
    ax.legend(loc='upper left');
    fig.savefig('doc/smoothingEffect-%s-%s.png' % (modelname, lab))
    